In [ ]:
!pip install qiskit
!pip install qiskit_algorithms
!pip install qiskit_ibm_runtime
!pip install nevergrad
!pip install pyDOE
!pip install scikit-quant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requi

In [ ]:
import numpy as np
import scipy
from qiskit.quantum_info import SparsePauliOp

In [ ]:
def construct_hamiltonian(j1, j2, grid):
  def nearest_neighbor(grid, i, j):
    i, j = i % len(grid[0]), j % len(grid)
    look_at = [[1, 0], [-1, 0], [0, 1], [0, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def next_nearest_neighbor(grid, i, j):
    look_at = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    result = []
    for element in look_at:
      dx, dy = element
      result.append([(i + dx) % len(grid[0]), (j + dy) % len(grid)])
    return result

  def generate_dot_product(grid, term, idxA, idxB):
    operation_template = ['I' for element in range(len(grid[0]) * len(grid))]
    dot_product = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
    for direction in ['X', 'Y', 'Z']:
      operation = operation_template
      operation[idxA], operation[idxB] = direction, direction
      dot_product += SparsePauliOp("".join(operation), coeffs=[term])
    return dot_product


  hamilonian = SparsePauliOp(('I' * len(grid[0]) * len(grid)), coeffs=[0])
  for i in range(len(grid[0])):
    for j in range(len(grid)):
      n_neighbors = nearest_neighbor(grid, i, j)
      nn_neighbors = next_nearest_neighbor(grid, i, j)

      for neighbor in n_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j1, idxA, idxB)
      for neighbor in nn_neighbors:
        idxA = (j * len(grid)) + i
        idxB = (neighbor[1] * len(grid)) + neighbor[0]
        hamilonian += generate_dot_product(grid, j2, idxA, idxB)

  return hamilonian.simplify()

from scipy.sparse.linalg import eigsh

In [ ]:
# Define the grid size
grid_size = (3,3)
grid = np.zeros(grid_size)

# Define interaction terms
j1 = 1.0  # Nearest-neighbor interaction strength
j2 = 0.5  # Next-nearest-neighbor interaction strength

# Construct the Hamiltonian
hamiltonian = construct_hamiltonian(j1, j2, grid).simplify()

# Print the Hamiltonian
print(hamiltonian)

SparsePauliOp(['XXIIIIIII', 'YYIIIIIII', 'ZZIIIIIII', 'XIXIIIIII', 'YIYIIIIII', 'ZIZIIIIII', 'XIIXIIIII', 'YIIYIIIII', 'ZIIZIIIII', 'XIIIIIXII', 'YIIIIIYII', 'ZIIIIIZII', 'XIIIXIIII', 'YIIIYIIII', 'ZIIIZIIII', 'XIIIIIIXI', 'YIIIIIIYI', 'ZIIIIIIZI', 'XIIIIXIII', 'YIIIIYIII', 'ZIIIIZIII', 'XIIIIIIIX', 'YIIIIIIIY', 'ZIIIIIIIZ', 'IIIXXIIII', 'IIIYYIIII', 'IIIZZIIII', 'IIIXIXIII', 'IIIYIYIII', 'IIIZIZIII', 'IIIXIIXII', 'IIIYIIYII', 'IIIZIIZII', 'IIIXIIIXI', 'IIIYIIIYI', 'IIIZIIIZI', 'IXIXIIIII', 'IYIYIIIII', 'IZIZIIIII', 'IIIXIIIIX', 'IIIYIIIIY', 'IIIZIIIIZ', 'IIXXIIIII', 'IIYYIIIII', 'IIZZIIIII', 'IIIIIIXXI', 'IIIIIIYYI', 'IIIIIIZZI', 'IIIIIIXIX', 'IIIIIIYIY', 'IIIIIIZIZ', 'IXIIIIXII', 'IYIIIIYII', 'IZIIIIZII', 'IIIIXIXII', 'IIIIYIYII', 'IIIIZIZII', 'IIXIIIXII', 'IIYIIIYII', 'IIZIIIZII', 'IIIIIXXII', 'IIIIIYYII', 'IIIIIZZII', 'IXXIIIIII', 'IYYIIIIII', 'IZZIIIIII', 'IXIIXIIII', 'IYIIYIIII', 'IZIIZIIII', 'IXIIIIIXI', 'IYIIIIIYI', 'IZIIIIIZI', 'IXIIIXIII', 'IYIIIYIII', 'IZIIIZIII', 'IXIIIIIIX

In [ ]:
# Get the Hamiltonian matrix
hamiltonian_matrix = hamiltonian.to_matrix(sparse=True)

# Calculate the eigenvalues using SciPy's eigsh
eigenvalues, eigenvectors = eigsh(hamiltonian_matrix, k=6, which='SA')  # Finding the 6 smallest eigenvalues

print("Eigenvalues:", eigenvalues)

Eigenvalues: [-27.876007 -27.876007 -27.876007 -27.876007 -27.876007 -27.876007]


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options


estimator = Estimator()

# Define interaction terms
j1 = 1.0  # Nearest-neighbor interaction strength
j2 = 0.5  # Next-nearest-neighbor interaction strength

grid_size = (3,3)
grid = np.zeros(grid_size)

# Construct the Hamiltonian
hamiltonian = construct_hamiltonian(j1, j2, grid).simplify()
print(hamiltonian)


# Define ansatz
from scipy.optimize import minimize

num_qubits = grid_size[0] * grid_size[1]
ansatz = qiskit.circuit.library.EfficientSU2(num_qubits, reps=1)
x0 = 2 * np.pi * np.random.random(ansatz.num_parameters)

def qcm_method(hamiltonian, qc):
    sample_size = 5000  # Adjust the sample size as needed
    # Measure Hamiltonian moments for H + λA and H - λA
    moments_plus = [0]
    moments_minus = [0]
    epsilon = 1e-1

    hamiltonian = hamiltonian.simplify()
    H_plus_lambda_k = (hamiltonian).simplify()
    upper = 5
    for k in range(1, upper):
        #tpb_sets, tpb_coeffs = find_optimal_tpb_sets([str(item) for item in H_plus_lambda_k.paulis], H_plus_lambda_k.coeffs)

        # Sample a subset of Pauli strings and coefficients
        if len(H_plus_lambda_k.paulis) > sample_size:
            sampled_paulis, sampled_coeffs = sample_pauli_strings(H_plus_lambda_k.paulis, H_plus_lambda_k.coeffs, sample_size)
            scaling_factor = np.linalg.norm(H_plus_lambda_k.coeffs) / np.linalg.norm(sampled_coeffs)
            H_plus_lambda_k = SparsePauliOp(sampled_paulis, [el * scaling_factor for el in sampled_coeffs])

        expectation = 0
        qcs = []
        #operators = []
        #for entries, coeffs in zip(tpb_sets, tpb_coeffs):
        #    operator = SparsePauliOp(entries[0], coeffs=[coeffs[0]])
        #    for i in range(1, len(entries)):
        #        operator += SparsePauliOp(entries[i], coeffs=[coeffs[i]])
        #    operators.append(operator)
        #    qcs.append(qc)
        #print(f"Running: {len(qcs)} Circuits to Compute TPB Pauli Strings")
        #expectation = sum(estimator.run(circuits=qcs, observables=operators).result().values)
        qc = qc
        H_copy = hamiltonian#.apply_layout(qc.layout)
        expectation = estimator.run(circuits=[qc], observables=[H_copy]).result().values[0]
        moments_plus.append(expectation)
        if(k + 1 < upper):
          H_plus_lambda_k = (H_plus_lambda_k @ hamiltonian).simplify()

    # Compute cumulants
    cumulants_plus = [0, moments_plus[1]]
    for n in range(2, upper):
        cumulant_plus = moments_plus[n] - sum([math.comb(n - 1, p) * cumulants_plus[p+1] * moments_plus[n - 1 - p] for p in range(n - 1)])
        cumulants_plus.append(cumulant_plus)

    # Compute energy estimates
    E_plus = cumulants_plus[1] - (cumulants_plus[2]**2 / (cumulants_plus[3]**2 - cumulants_plus[2] * cumulants_plus[4])) * (np.sqrt(3 * cumulants_plus[3]**2 - 2 * cumulants_plus[2] * cumulants_plus[4]) - cumulants_plus[3])
    return E_plus
# QCM
#observable_estimate = qcm_method(hamiltonian, neel_ham, ansatz)
#print("Estimated Observable:", observable_estimate)

SparsePauliOp(['XXIIIIIII', 'YYIIIIIII', 'ZZIIIIIII', 'XIXIIIIII', 'YIYIIIIII', 'ZIZIIIIII', 'XIIXIIIII', 'YIIYIIIII', 'ZIIZIIIII', 'XIIIIIXII', 'YIIIIIYII', 'ZIIIIIZII', 'XIIIXIIII', 'YIIIYIIII', 'ZIIIZIIII', 'XIIIIIIXI', 'YIIIIIIYI', 'ZIIIIIIZI', 'XIIIIXIII', 'YIIIIYIII', 'ZIIIIZIII', 'XIIIIIIIX', 'YIIIIIIIY', 'ZIIIIIIIZ', 'IIIXXIIII', 'IIIYYIIII', 'IIIZZIIII', 'IIIXIXIII', 'IIIYIYIII', 'IIIZIZIII', 'IIIXIIXII', 'IIIYIIYII', 'IIIZIIZII', 'IIIXIIIXI', 'IIIYIIIYI', 'IIIZIIIZI', 'IXIXIIIII', 'IYIYIIIII', 'IZIZIIIII', 'IIIXIIIIX', 'IIIYIIIIY', 'IIIZIIIIZ', 'IIXXIIIII', 'IIYYIIIII', 'IIZZIIIII', 'IIIIIIXXI', 'IIIIIIYYI', 'IIIIIIZZI', 'IIIIIIXIX', 'IIIIIIYIY', 'IIIIIIZIZ', 'IXIIIIXII', 'IYIIIIYII', 'IZIIIIZII', 'IIIIXIXII', 'IIIIYIYII', 'IIIIZIZII', 'IIXIIIXII', 'IIYIIIYII', 'IIZIIIZII', 'IIIIIXXII', 'IIIIIYYII', 'IIIIIZZII', 'IXXIIIIII', 'IYYIIIIII', 'IZZIIIIII', 'IXIIXIIII', 'IYIIYIIII', 'IZIIZIIII', 'IXIIIIIXI', 'IYIIIIIYI', 'IZIIIIIZI', 'IXIIIXIII', 'IYIIIYIII', 'IZIIIZIII', 'IXIIIIIIX

In [ ]:
iterations = 1

new_estimator = qiskit.primitives.Estimator()
def cost_func(params, ansatz, hamiltonian, estimator):
    global iterations
    qc = ansatz.assign_parameters(params)
    #qc = pm.run(qc)
    #H_copy = hamiltonian.apply_layout(qc.layout)
    H_copy = hamiltonian
    iterations += 1
    result = estimator.run(circuits=[qc], observables=[H_copy]).result()
    energy = np.real((result.values[0]))
    #if(iterations > 350):
    #energy = qcm_method(hamiltonian, qc)
    print(f"Iteration {iterations}: Energy = {energy}")
    return energy

res = minimize(cost_func,x0,args=(ansatz, hamiltonian, new_estimator), method="cobyla",options={"maxiter": 600})

Iteration 2: Energy = 2.5849955463727676
Iteration 3: Energy = 0.30654064112318236
Iteration 4: Energy = -2.1333383194627173
Iteration 5: Energy = -2.4029874441011865
Iteration 6: Energy = -4.109316066061129
Iteration 7: Energy = -5.388917600761304
Iteration 8: Energy = -4.967075523589254
Iteration 9: Energy = -5.54764139349011
Iteration 10: Energy = -6.188860180213608
Iteration 11: Energy = -3.6227592981091226
Iteration 12: Energy = -6.930645969046184
Iteration 13: Energy = -6.090491595906661
Iteration 14: Energy = -6.9285745024167
Iteration 15: Energy = -6.851834797496491
Iteration 16: Energy = -5.806621699569733
Iteration 17: Energy = -6.886914018783078
Iteration 18: Energy = -6.388805663558362
Iteration 19: Energy = -5.876894210111221
Iteration 20: Energy = -5.721806019425156
Iteration 21: Energy = -7.781372957097809
Iteration 22: Energy = -6.332768706244539
Iteration 23: Energy = -7.7211108019815455
Iteration 24: Energy = -7.333783408317533
Iteration 25: Energy = -6.98811554871643

In [ ]:
def neel_order(dim):
    L = dim[0] * dim[1]
    neel_op = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        x, y = i % dim[0], i // dim[0]
        sign = (-1) ** (x + y)
        label = ['I'] * L
        label[i] = 'Z'
        neel_op += SparsePauliOp(''.join(label), coeffs=[sign])

    neel_op = neel_op.simplify()
    neel_op /= L  # Normalize the Néel operator

    return (neel_op @ neel_op).simplify()

def dimer_order(dim):
    Lx, Ly = dim
    num_spins = Lx*Ly
    dimer_op = SparsePauliOp(('I' * num_spins), coeffs=[0])
    normalization = 0

    for x in range(0, Lx//2, 2):
        for y in range(Ly):
            i = y * Ly + x
            j = ((x + 1) % Lx) + y * Ly
            sign = (-1)**(x)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for x in range(0, Lx//2, 3):
        for y in range(Ly):
            i = y * Ly + x
            j = ((x + 2) % Lx) + y * Ly
            sign = (-1)**(x)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for y in range(0, Ly//2, 2):
        for x in range(Lx):
            i = y * Ly + x
            j = ((y + 1) % Ly)*Ly + x
            sign = (-1)**(y)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()

    for y in range(0, Ly//2, 3):
        for x in range(Lx):
            i = y * Ly + x
            j = ((y + 2) % Ly)*Ly + x
            sign = (-1)**(y)
            label = ['I'] * num_spins
            label[i] = label[j] = 'X'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Y'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            label[i] = label[j] = 'Z'
            dimer_op += sign * SparsePauliOp(''.join(label), coeffs=[sign])
            normalization += 1

    dimer_op.simplify()
    dimer_op /= normalization
    return (dimer_op @ dimer_op).simplify()

def spin_order(dim):
    L = dim[0] * dim[1]
    spin_corr = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        for j in range(i+1, L):
            x1, y1 = i % dim[0], i // dim[0]
            x2, y2 = j % dim[0], j // dim[0]

            if abs(x1 - x2) + abs(y1 - y2) == 1:  # Nearest neighbors
                label = ['I'] * L
                label[i] = 'Z'
                label[j] = 'Z'
                spin_corr += SparsePauliOp(''.join(label), coeffs=[1])

    spin_corr = spin_corr.simplify()
    spin_corr /= L  # Normalize the spin correlation operator

    return spin_corr

def spin_correlation(dim, max_distance=None):
    L = dim[0] * dim[1]
    spin_corr = SparsePauliOp(('I' * L), coeffs=[0])

    for i in range(L):
        for j in range(i+1, L):
            x1, y1 = i % dim[0], i // dim[0]
            x2, y2 = j % dim[0], j // dim[0]

            distance = abs(x1 - x2) + abs(y1 - y2)
            if max_distance is None or distance <= max_distance:
                label = ['I'] * L
                label[i] = 'Z'
                label[j] = 'Z'
                coeff = 1 / (distance ** 2)  # Weight coefficient by inverse square of distance
                spin_corr += SparsePauliOp(''.join(label), coeffs=[coeff])

    spin_corr = spin_corr.simplify()
    spin_corr /= L  # Normalize the spin correlation operator

    return spin_corr

dim = [3,3]
neel_ham = neel_order(dim)
dimer_ham = dimer_order(dim)
spin_corr_local = spin_order(dim)
spin_corr_global = spin_correlation(dim)

In [ ]:
from qiskit.circuit.library import CXGate, CYGate, CZGate
from qiskit.circuit import QuantumCircuit

def make_controlled_pauli_gate(pauli_string, control_qubit=0):
    """
    Create a controlled gate for the given Pauli string.

    Args:
        pauli_string (str): A string representing Pauli operations on each qubit (e.g., 'XXXIIIIXYZ').
        control_qubit (int): The index of the control qubit.

    Returns:
        QuantumCircuit: A quantum circuit with the controlled Pauli gates.
    """
    num_qubits = len(pauli_string)
    qc = QuantumCircuit(num_qubits + 1)  # +1 for the control qubit

    for target_qubit, pauli in enumerate(pauli_string):
        if pauli == 'X':
            qc.cx(control_qubit, target_qubit + 1)
        elif pauli == 'Y':
            qc.cy(control_qubit, target_qubit + 1)
        elif pauli == 'Z':
            qc.cz(control_qubit, target_qubit + 1)
        elif pauli != 'I':
            raise ValueError(f"Unsupported Pauli term: {pauli}")

    return qc.to_gate()

class LCU_estimator:
    def __init__(self, estimator):
      self.base_estimator = estimator

    def deconstruct_operator(self, hamiltonian):
      return [str(item) for item in hamiltonian.paulis], hamiltonian.coeffs

    def make_operators(self, paulis):
      return [make_controlled_pauli_gate(pauli) for pauli in paulis]

    def normalize_hamiltonian(self, hamiltonian):
      hamiltonian = hamiltonian.simplify()
      hamiltonian_mag = np.sum(np.abs(hamiltonian.coeffs).real)
      hamiltonian.coeffs = [el / hamiltonian_mag for el in hamiltonian.coeffs]
      return hamiltonian

    def run(self, circuits, observables):
      results = []
      num_indicies = 2

      for circuit, observable in zip(circuits, observables):
        observable = self.normalize_hamiltonian(observable)
        observable_paulis, observable_coeffs = self.deconstruct_operator(observable)
        observable_operators = self.make_operators(observable_paulis)

        total_sum = sum(np.abs(observable_coeffs))
        probabilities = [np.abs(coeff) / total_sum for coeff in observable_coeffs]

        mean_value = 0.0
        last_value = 1e30
        iterations = 0
        percision = 1e-3
        circuits_trial = []
        observables_trial = []
        while(np.abs(mean_value - last_value) > percision and not(iterations > 2)):
          indicies = np.random.choice(len(observable_coeffs), size=num_indicies, replace=False, p=probabilities)

          qc = QuantumCircuit(circuit.num_qubits + 1)
          qc.h(0)
          qc.append(circuit, range(1,circuit.num_qubits+1))
          for index in indicies:
            qc.append(observable_operators[indicies[0]], range(0,circuit.num_qubits+1))
            qc.append(observable_operators[indicies[1]], range(0,circuit.num_qubits+1))

          observable_combined = SparsePauliOp("X" + "Z" * circuit.num_qubits, coeffs=[1])

          # Measure X on the control
          circuits_trial.append(qc)
          observables_trial.append(observable_combined)
          iterations += 1

          if(iterations % 50 == 0):
            last_value = mean_value
            mean_value = mean_value*((iterations - 50)/iterations) + (50/iterations)*(sum(self.base_estimator.run(circuits=circuits_trial, observables=observables_trial).result().values)/50)
            circuits_trial = []
            observables_trial = []

          if(np.abs(mean_value - last_value) < percision):
            print(f"Took {iterations} iterations until observable convergence")
        results.append(mean_value)

      return results

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.primitives import Estimator
import math

import networkx as nx


def commutes(pauli1, pauli2):
    """
    Check if two Pauli strings commute.
    """
    non_identity_indices = [i for i in range(len(pauli1)) if pauli1[i] != 'I' and pauli2[i] != 'I']
    non_identity_differences = sum(pauli1[i] != pauli2[i] for i in non_identity_indices)
    return non_identity_differences % 2 == 0

def find_cliques_heuristic(graph):
    """
    Find cliques using a greedy heuristic.
    """
    cliques = []
    nodes = list(graph.nodes())

    while nodes:
        node = nodes.pop()
        neighbors = list(graph.neighbors(node))
        clique = [node]

        for neighbor in neighbors:
            if all(graph.has_edge(neighbor, member) for member in clique):
                clique.append(neighbor)

        cliques.append(clique)
        nodes = [n for n in nodes if n not in clique]

    return cliques

def find_optimal_tpb_sets(pauli_strings, coeffs, sample_size=None):
    if sample_size and sample_size < len(pauli_strings):
        pauli_strings, coeffs = sample_pauli_strings(pauli_strings, coeffs, sample_size)

    # Create an adjacency matrix where edges represent commutation
    n = len(pauli_strings)
    adjacency_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(i + 1, n):
            if commutes(pauli_strings[i], pauli_strings[j]):
                adjacency_matrix[i, j] = True
                adjacency_matrix[j, i] = True

    # Create a graph from the adjacency matrix
    G = nx.Graph(adjacency_matrix)

    # Find cliques using the greedy heuristic
    cliques = find_cliques_heuristic(G)

    # Sort cliques by size in descending order
    cliques.sort(key=len, reverse=True)

    # Assign Pauli strings to TPB sets
    tpb_sets = []
    used = set()
    for clique in cliques:
        tpb_set = []
        for idx in clique:
            if idx not in used:
                tpb_set.append(pauli_strings[idx])
                used.add(idx)
        if tpb_set:
            tpb_sets.append(tpb_set)

    # Reconstruct the coefficients for the TPB sets
    tpb_coeffs = []
    for tpb_set in tpb_sets:
        set_coeffs = []
        for p in tpb_set:
            idx = pauli_strings.index(p)
            set_coeffs.append(coeffs[idx])
        tpb_coeffs.append(set_coeffs)

    return tpb_sets, tpb_coeffs

def sample_pauli_strings(pauli_strings, coeffs, sample_size):
    """
    Sample a subset of the Pauli strings and their coefficients with a bias towards the beginning of the list.
    """
    n = len(pauli_strings)
    weights = np.linspace(1, 0.1, n)  # Linear weights decreasing from 1 to 0.1
    weights /= weights.sum()  # Normalize to sum to 1

    indices = np.random.choice(n, size=sample_size, replace=False, p=weights)
    sampled_paulis = [pauli_strings[i] for i in indices]
    sampled_coeffs = [coeffs[i] for i in indices]
    return sampled_paulis, sampled_coeffs

def qcm_method(hamiltonian, qc):
    sample_size = 1e30  # Adjust the sample size as needed
    # Measure Hamiltonian moments for H + λA and H - λA
    moments_plus = [0]
    epsilon = 1e-3

    hamiltonian = hamiltonian.simplify()
    H_plus_lambda_k = (hamiltonian).simplify()
    upper = 5
    for k in range(1, upper):
        print(f"Computing for order {k}... Pauli Strings: {len(H_plus_lambda_k.paulis)}")

        # Sample a subset of Pauli strings and coefficients
        if len(H_plus_lambda_k.paulis) > sample_size:
            sampled_paulis, sampled_coeffs = sample_pauli_strings(H_plus_lambda_k.paulis, H_plus_lambda_k.coeffs, sample_size)
            scaling_factor = np.linalg.norm(H_plus_lambda_k.coeffs) / np.linalg.norm(sampled_coeffs)
            H_plus_lambda_k = SparsePauliOp(sampled_paulis, [el * scaling_factor for el in sampled_coeffs]).simplify()

        expectation = estimator.run(circuits=qcs, observables=operators)
        print("K: ", k, " ", expectation)
        moments_plus.append(expectation)
        if(k + 1 < upper):
          H_plus_lambda_k = (H_plus_lambda_k @ hamiltonian).simplify()

    # Compute cumulants
    cumulants_plus = [0, moments_plus[1]]
    for n in range(2, upper):
        cumulant_plus = moments_plus[n] - sum([math.comb(n - 1, p) * cumulants_plus[p+1] * moments_plus[n - 1 - p] for p in range(n - 1)])
        cumulants_plus.append(cumulant_plus)

    # Compute energy estimates
    E_plus = cumulants_plus[1] - (cumulants_plus[2]**2 / (cumulants_plus[3]**2 - cumulants_plus[2] * cumulants_plus[4])) * (np.sqrt(3 * cumulants_plus[3]**2 - 2 * cumulants_plus[2] * cumulants_plus[4]) - cumulants_plus[3])
    return E_plus

In [ ]:
print(estimator)

In [ ]:
qc_copy = pm.run(ansatz.assign_parameters(res.x))
hamiltonian_copy = hamiltonian.apply_layout(qc_copy.layout)
print(estimator.run(circuits=[qc_copy], observables=[hamiltonian_copy]).result().values)

[-19.48457947]


In [ ]:
observable_estimate = qcm_method(hamiltonian.simplify(), ansatz.assign_parameters(res.x))
print("Estimated Observable:", observable_estimate)

Computing for order 1... Pauli Strings: 108
Running: 3 Circuits to Compute TPB Pauli Strings
K:  1   -19.62968053686153
Computing for order 2... Pauli Strings: 2755
Running: 359 Circuits to Compute TPB Pauli Strings


KeyboardInterrupt: 

In [ ]:
# Compute the expectation value
hamiltonian_matrix = hamiltonian.to_matrix(sparse=True)
expectation_value = np.dot(eigenvectors[:, 0].conj().T, hamiltonian_matrix.dot(eigenvectors[:, 0]))

print("Expectation value:", expectation_value)

Expectation value: (-28.65120839965672+8.881784197001252e-16j)


In [ ]:
j2_j1s = [0, 0.2, 0.4, 0.5, 0.56, 0.8, 1.0]
for j2_j1 in j2_j1s:
  # Define interaction terms
  j1 = 1.0  # Nearest-neighbor interaction strength
  j2 = j2_j1  # Next-nearest-neighbor interaction strength

  grid_size = (3,3)
  grid = np.zeros(grid_size)

  # Construct the Hamiltonian
  hamiltonian = construct_hamiltonian(j1, j2, grid).simplify()
  ansatz = qiskit.circuit.library.TwoLocal(num_qubits, 'rx', 'cz', reps=3)
  x0 = 2 * np.pi * np.random.random(ansatz.num_parameters)
  res = minimize(cost_func,x0,args=(ansatz, hamiltonian, new_estimator), method="cobyla",options={"maxiter": 800})
  qc = ansatz.assign_parameters(res.x)
  qc = pm.run(qc)
  H_copy = hamiltonian.apply_layout(qc.layout)
  before = estimator.run(circuits=[qc], observables=[H_copy]).result().values[0]

  hamiltonian_matrix = hamiltonian.to_matrix(sparse=True)
  eigenvalues, _ = eigsh(hamiltonian_matrix, k=1, which='SA')  # Finding the 6 smallest eigenvalues

  after = qcm_method(hamiltonian, ansatz.assign_parameters(res.x))
  print(f"J2/J1: {j2}, Ground Truth: {eigenvalues[0]}, Before Correction: {before}, After Quantum Moments: {after}")

Computing for order 1... Pauli Strings: 54
Computing for order 2... Pauli Strings: 919
Computing for order 3... Pauli Strings: 6354
Computing for order 4... Pauli Strings: 19306
J2/J1: 0, Ground Truth: -31.752013999428137, Before Correction: -24.74993272993075, After Quantum Moments: -29.629457578762974
Computing for order 1... Pauli Strings: 108
Computing for order 2... Pauli Strings: 2755
Computing for order 3... Pauli Strings: 18127
Computing for order 4... Pauli Strings: 32896
J2/J1: 0.2, Ground Truth: -30.201611199542466, Before Correction: -22.192335833303584, After Quantum Moments: -29.09614317854088
Computing for order 1... Pauli Strings: 108
Computing for order 2... Pauli Strings: 2755
Computing for order 3... Pauli Strings: 18127
Computing for order 4... Pauli Strings: 32896
J2/J1: 0.4, Ground Truth: -28.6512083996569, Before Correction: -23.12841623257085, After Quantum Moments: -28.35853373225674
Computing for order 1... Pauli Strings: 108
Computing for order 2... Pauli Str